In [1]:
from collections import Counter
import random

import numpy as np
import pyarrow.dataset as ds
import duckdb
import pandas as pd
import numpy as np
import seaborn as sns
import plotly as pl
import plotly.express as px
import json

INDICADORES = {
    "AFD": "Adequação da Formação Docente",
    "ICG": "Complexidade de Gestão da Escola",
    "IED": "Esforço Docente",
    "ATU": "Média de Alunos por Turma",
    "HAD": "Média de Horas-aula diária",
    "DSU": "Percentual de Docentes com Curso Superior",
    "TDI": "Taxas de Distorção Idade-série"
}

DIMENSIONS = {
    "Ano": "NU_ANO_CENSO",
    "Nome da região geográfica": "NO_REGIAO",
    "Nome da Unidade da Federação": "NO_UF",
    "Nome da Mesorregião": "NO_MESORREGIAO",
    "Nome da Microrregião": "NO_MICRORREGIAO"
}

con = duckdb.connect('../../file.db')

microdados = ds.dataset("../../data/transformed/microdados.parquet", format="parquet", partitioning="hive")
con.register("microdados", microdados)

indicadores = {}
for indicador in INDICADORES:
    indicadores[indicador] = ds.dataset(f"../../data/transformed/indicadores/{indicador}.parquet", format="parquet", partitioning="hive")
    con.register(indicador, indicadores[indicador])

In [28]:
con.execute(f"""
    select
        *
        from TDI
        WHERE TP_GRUPO = 'FUN_01_CAT_0' and METRICA != 0 and METRICA is not null and NU_ANO_CENSO=2021
        limit 10
    """
).df()

,NO_REGIAO,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,CO_ENTIDADE,NO_CATEGORIA,NO_DEPENDENCIA,GRUPO,METRICA,NU_ANO_CENSO
0,Norte,RO,1100015,Alta Floresta D'Oeste,11025077,Urbana,Municipal,FUN_01_CAT_0,14.3,2021
1,Norte,RO,1100015,Alta Floresta D'Oeste,11025280,Urbana,Municipal,FUN_01_CAT_0,11.1,2021
2,Norte,RO,1100015,Alta Floresta D'Oeste,11025654,Urbana,Municipal,FUN_01_CAT_0,0.8,2021
3,Norte,RO,1100015,Alta Floresta D'Oeste,11037962,Rural,Estadual,FUN_01_CAT_0,33.3,2021
4,Norte,RO,1100015,Alta Floresta D'Oeste,11038047,Rural,Estadual,FUN_01_CAT_0,50.0,2021
5,Norte,RO,1100015,Alta Floresta D'Oeste,11038373,Rural,Estadual,FUN_01_CAT_0,50.0,2021
6,Norte,RO,1100015,Alta Floresta D'Oeste,11046236,Rural,Estadual,FUN_01_CAT_0,66.7,2021
7,Norte,RO,1100015,Alta Floresta D'Oeste,11046244,Rural,Estadual,FUN_01_CAT_0,50.0,2021
8,Norte,RO,1100015,Alta Floresta D'Oeste,11046252,Rural,Estadual,FUN_01_CAT_0,30.0,2021
9,Norte,RO,1100023,Ariquemes,11006854,Urbana,Municipal,FUN_01_CAT_0,9.1,2021


In [20]:
con.execute(f"""
    select
        TP_GRUPO,
        METRICA
        from TDI
        where NU_ANO_CENSO=2018 and CO_ENTIDADE='11025077'
        order by 1
    """
).df()

,GRUPO,METRICA
0,TDI_F00,0.0
1,TDI_F01,0.0
2,TDI_F02,0.0
3,TDI_F03,21.4
4,TDI_F04,40.0
5,TDI_F05,50.0
6,TDI_F06,14.3
7,TDI_F07,40.0
8,TDI_F08,16.7
9,TDI_F14,17.5


In [21]:
con.execute(f"""
    select
        distinct(TP_GRUPO)
        from TDI
        where NU_ANO_CENSO=2018
        order by 1
    """
).df().TP_GRUPO

0     TDI_F00
1     TDI_F01
2     TDI_F02
3     TDI_F03
4     TDI_F04
5     TDI_F05
6     TDI_F06
7     TDI_F07
8     TDI_F08
9     TDI_F14
10    TDI_F58
11    TDI_FUN
12    TDI_M01
13    TDI_M02
14    TDI_M03
15    TDI_M04
16    TDI_MED
Name: GRUPO, dtype: object